In [46]:
import matplotlib.pyplot as plt
from matplotlib import gridspec
import numpy as np
import scipy.io
import tensorflow as tf
import tensorflow_addons as tfa
import pandas as pd
import os

from tensorflow import keras

from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, LSTM, Conv2D
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [47]:
tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [48]:
label=pd.read_csv('Forest Segmented\meta_data.csv')

classes = {
    '0': (0, 0, 0,),
    '1': (255, 255, 255)
}

print(label)

path_original_images = 'Forest Segmented/images'
path_label_images = 'Forest Segmented/masks'
num_class = 2

print(num_class)

                  image                mask
0      10452_sat_08.jpg   10452_mask_08.jpg
1      10452_sat_18.jpg   10452_mask_18.jpg
2     111335_sat_00.jpg  111335_mask_00.jpg
3     111335_sat_01.jpg  111335_mask_01.jpg
4     111335_sat_02.jpg  111335_mask_02.jpg
...                 ...                 ...
5103  998002_sat_31.jpg  998002_mask_31.jpg
5104  998002_sat_32.jpg  998002_mask_32.jpg
5105  998002_sat_40.jpg  998002_mask_40.jpg
5106  998002_sat_41.jpg  998002_mask_41.jpg
5107  998002_sat_50.jpg  998002_mask_50.jpg

[5108 rows x 2 columns]
2


In [ ]:
def shuffle(arr1, arr2):
    randomize = np.arange(len(arr1))
    np.random.shuffle(randomize)
    return arr1[randomize], arr2[randomize]

In [50]:
img, msk = label.items()
image, mask = shuffle(np.array(img[1]), np.array(msk[1]))
print(image)
print(mask)

['848728_sat_52.jpg' '122104_sat_22.jpg' '830444_sat_45.jpg' ...
 '277994_sat_53.jpg' '352808_sat_04.jpg' '810368_sat_64.jpg']
['848728_mask_52.jpg' '122104_mask_22.jpg' '830444_mask_45.jpg' ...
 '277994_mask_53.jpg' '352808_mask_04.jpg' '810368_mask_64.jpg']


In [51]:
print(image.shape[0])
print(mask.shape[0])

5108
5108


In [52]:
def get_original_images():
    temp_img = []
    for a in range(image.shape[0]):
        temp_img.append(tf.keras.preprocessing.image.img_to_array(tf.keras.utils.load_img(f"{path_original_images}/{image[a]}", target_size=(128,128))))
    
    return np.array(temp_img)

In [53]:
def get_mod_images(mod_path):
    temp_img = []
    for a in range(image.shape[0]):
        temp_img.append(tf.keras.preprocessing.image.img_to_array(tf.keras.utils.load_img(f"{mod_path}/{image[a]}", target_size=(128,128))))
    
    return np.array(temp_img)

In [54]:
def classconvert():
    rtn = []
    for a in range(mask.shape[0]):
        img_mask = tf.keras.preprocessing.image.img_to_array(tf.keras.utils.load_img(f"{path_label_images}/{mask[a]}", target_size=(128,128)))
        temp_mask = np.zeros(shape=(img_mask.shape[0], img_mask.shape[1]), dtype = np.uint32)

        for row in range(img_mask.shape[0]):
            for col in range(img_mask.shape[1]):
                if img_mask[row, col, 0] > 128:
                    temp_mask[row, col] = 1
        temp_mask = np.reshape(temp_mask, (temp_mask.shape[0], temp_mask.shape[1], 1))
        rtn.append(temp_mask)
    return np.array(rtn)

In [55]:
x = get_original_images()
y = classconvert()
print(x.shape)
print(y.shape)
np.save('x.npy',x)
np.save('y.npy',y)

(5108, 128, 128, 3)
(5108, 128, 128, 1)


In [56]:
noise = get_mod_images('mod_datasets/gnoise')
print(noise.shape)

(5108, 128, 128, 3)


In [57]:
edge = get_mod_images('mod_datasets/edge')
print(edge.shape)

(5108, 128, 128, 3)


In [58]:
hist = get_mod_images('mod_datasets/histo')
print(hist.shape)

(5108, 128, 128, 3)


In [59]:
edge_noise = get_mod_images('mod_datasets/edge_gnoise')
print(edge_noise.shape)

(5108, 128, 128, 3)


In [60]:
edge_hist = get_mod_images('mod_datasets/edge_histo')
print(edge_hist.shape)

(5108, 128, 128, 3)


In [61]:
hist_noise = get_mod_images('mod_datasets/gnoise_histo')
print(hist_noise.shape)

(5108, 128, 128, 3)


In [62]:
all = get_mod_images('mod_datasets/all')
print(all.shape)

(5108, 128, 128, 3)


In [63]:
np.save("noise.npy", noise)
np.save("edge.npy", edge)
np.save("hist.npy", hist)
np.save("edge_noise.npy", edge_noise)
np.save("edge_hist.npy", edge_hist)
np.save("hist_noise.npy", hist_noise)
np.save("all.npy", all)

In [64]:
noise = np.load("noise.npy",)
edge = np.load("edge.npy")
hist = np.load("hist.npy")
edge_noise = np.load("edge_noise.npy")
edge_hist = np.load("edge_hist.npy")
hist_noise = np.load("hist_noise.npy")
all  = np.load("all.npy")

In [65]:
import random
random_indexes = random.sample(range(len(noise)), int(len(noise) * 0.2))
print(len(random_indexes))

1021


In [66]:
import random

def generate_dataset(dataset, noiseset, noiselevel = 0.5):
    temp = []
    random_indexes = random.sample(range(len(dataset)), int(len(dataset) * noiselevel))
    for i in range(len(dataset)):
        if i in random_indexes:
            temp.append(noiseset[i])
        else:
            temp.append(dataset[i])
    return np.array(temp)

In [67]:
original_dataset = get_original_images()
dataset_noise = generate_dataset(original_dataset[0:4086], noise[0:4086])


dataset_edge_noise = generate_dataset(edge[0:4086], edge_noise[0:4086])

dataset_hist_noise = generate_dataset(hist[0:4086], hist_noise[0:4086])
dataset_all = generate_dataset(edge_hist[0:4086], all[0:4086])
print(dataset_all.shape)

(4086, 128, 128, 3)


In [68]:
np.save("dataset_noise.npy", dataset_noise)
np.save("dataset_edge_noise.npy", dataset_edge_noise)
np.save("dataset_hist_noise.npy", dataset_hist_noise)
np.save("dataset_all.npy", dataset_all)